# Coleta de Dados DEX via Bitquery – raw zone

Este notebook executa a coleta de dados históricos de transações realizadas em exchanges descentralizadas (DEX) usando a **API GraphQL da Bitquery**. Os dados são extraídos por rede e período de forma incremental e armazenados na **camada raw** do Data Lake, no formato Delta.

---

## Objetivo

Construir uma camada **raw** de transações DEX acima de $1.000, com foco em:

- Persistência de dados brutos organizados
- Criação futura de dimensões e fato
- Possibilidade de reprocessamento

---

## Estrutura do pipeline

1. **Intervalo de coleta**  
   Os dados são extraídos dos últimos **45 dias** até a data atual (`hoje`).

2. **Chunks de tempo**
   O Bitquery limita a resposta a ~10000 linhas por chamada. Para evitar truncamentos, o script divide o intervalo em **blocos de 5 dias** (`chunk_size`).

3. **Loop por rede**
   As redes consultadas são:
   - `ethereum`
   - `matic` (Polygon)
   - `fantom`
   - `avalanche`
   - `klaytn`

4. **Requisição à API GraphQL**
   Para cada rede e chunk, é enviada uma query com o filtro:
   ```graphql
   tradeAmountUsd: { gt: 1000 }
   ```

5. **Processamento**
   - A resposta JSON da API é transformada em `pandas.DataFrame`
   - Em seguida, convertida para `Spark DataFrame` com os campos relevantes
   - Cada batch é salvo com `append` na tabela Delta `workspace.ethereum.dex_trades_raw`

---

## Campos extraídos

Os principais atributos da transação incluem:

- `date`, `network`, `protocol`, `side`
- `baseCurrency`, `quoteCurrency`, `buyCurrency`, `sellCurrency`
- `tradeAmount`, `buyAmount`, `sellAmount`, `baseAmount`
- `quotePrice`, `price`, `trades`
- `gas`, `gasPrice`, `gasValue`
- `exchange`, `smartContract`

---

## Boas práticas aplicadas

-  **Paginação manual via chunk temporal**
-  **Tratamento de exceções por rede**
-  **Inclusão de metadados (rede, período de coleta)**
-  **Armazenamento incremental em Delta Lake**

---

## Próximos passos

-  Construção da **Bronze Layer** com deduplicação e normalização de colunas aninhadas
-  Modelagem dimensional (Star Schema com dimensões e fato)
-  Desenvolvimento de painéis analíticos e cálculos de volume de liquidez por protocolo e rede

---


## Código da Extração

In [0]:
import requests
import json
from datetime import datetime, timedelta
from web3 import Web3
import pandas as pd

hoje = datetime.utcnow()
inicio_total = hoje - timedelta(days=45)
fim_total = hoje

print(f"Coleta do período: {inicio_total.strftime('%Y-%m-%d')} até {fim_total.strftime('%Y-%m-%d')}")

usdc_checksum = Web3.to_checksum_address("0xA0b86991c6218b36c1d19d4a2e9Eb0cE3606eB48")
#Token Esgotado, sem problemas deixar exposto.
access_token = "ory_at_OgMtBmjDZ3HmRMpaAhnGsmJt-uSowszUQk3zDLfI4_s.62SE6S-mnHowojFNXOWv7T77l-jxtqMWuO4iMDZ_hIA"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}
networks = ["ethereum", "matic", "fantom", "avalanche", "klaytn"]
url = "https://graphql.bitquery.io"

#Chunk de 1 dia, pois a API trava com mais dias em gt:1000
chunk_size = timedelta(days=5)
data_atual = inicio_total

while data_atual < fim_total:
    inicio_chunk = data_atual
    fim_chunk = min(data_atual + chunk_size, fim_total)

    str_inicio = inicio_chunk.strftime("%Y-%m-%d")
    str_fim = fim_chunk.strftime("%Y-%m-%d")

    print(f"\n Coletando dados de {str_inicio} até {str_fim}")

    dfs = []

    for network in networks:
        print(f"🔍 Coletando dados para: {network}")

        query = f'''
        {{
            ethereum(network: {network}) {{
                dexTrades(
                    date: {{after: "{str_inicio}", till: "{str_fim}"}},
                    tradeAmountUsd: {{gt: 1000}},
                    options: {{limit: 1000}}
                ) {{
                    date {{ date }}
                    baseCurrency {{ symbol name address }}
                    quoteCurrency {{ symbol name address }}
                    buyCurrency {{ symbol }}
                    sellCurrency {{ symbol name address tokenId }}
                    exchange {{ name fullName }}
                    tradeAmount(in: USD)
                    buyAmount
                    sellAmount
                    baseAmount
                    quotePrice
                    price
                    trades: count
                    protocol
                    side
                    gas
                    gasPrice
                    gasValue
                    smartContract {{
                        contractType
                        currency {{ name symbol }}
                    }}
                }}
            }}
        }}
        '''

        try:
            response = requests.post(url, headers=headers, data=json.dumps({"query": query}))
            print("Status:", response.status_code)
            trades = response.json().get("data", {}).get("ethereum", {}).get("dexTrades", [])

            if trades:
                df = pd.DataFrame(trades)
                df["network"] = network
                df["data_inicio"] = str_inicio
                df["data_fim"] = str_fim
                dfs.append(df)
            else:
                print(f"Nenhum dado retornado para {network} ({str_inicio} até {str_fim})")
        except Exception as e:
            print(f"Erro ao coletar dados para {network}: {e}")

    if dfs:
        df_total = pd.concat(dfs, ignore_index=True)

        colunas_permitidas = [
            "date", "baseCurrency", "quoteCurrency", "buyCurrency", "sellCurrency",
            "exchange", "tradeAmount", "buyAmount", "sellAmount", "baseAmount",
            "quotePrice", "price", "trades", "protocol", "side", "gas",
            "gasPrice", "gasValue", "smartContract", "network"
        ]
        df_total = df_total[colunas_permitidas] 

        df_spark = spark.createDataFrame(df_total)


        df_spark.write \
            .format("delta") \
            .mode("append") \
            .saveAsTable("workspace.ethereum.dex_trades_raw")

        print(f"Dados salvos para o período {str_inicio} até {str_fim}")
    else:
        print("Nenhum dado coletado nesse chunk")

    data_atual = fim_chunk